## Libraries

In [2]:
import os
# from keras_unet_collection import models, utils
import tensorflow as tf
import cv2
import keras
import segmentation_models as sm
import numpy as np
import matplotlib.pyplot as plt
import gdown
import datetime
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import requests
# Notifications config:
url_notif = r'https://api.pushcut.io/nijldnK5Ud5uQXRJI0v_G/notifications/Training%20ended'
AUTOTUNE = tf.data.experimental.AUTOTUNE

Segmentation Models: using `keras` framework.


In [3]:
tf.__version__

'2.10.0'

In [24]:
import numpy as np
fold_att = np.load('kfold/Final_ablation/attn_unet_orig.npy')
fold_unet = np.load('kfold/Final_ablation/attn_unet_aug4.npy')
# fold_aug2 = np.load('kfold/unet_5fold_bright_contr.npy')

print(np.mean(fold_att, axis=0))
print(np.mean(fold_unet, axis=0))
# print(np.mean(fold_aug, axis=0))
# print(np.mean(fold_aug2, axis=0))
# print(fold_aug2)

[0.01079666 0.7355445  0.8303529 ]
[0.01101765 0.72712053 0.8226809 ]


In [25]:
uq_results = np.load('Uncertainty_comparison/Uncertainty_comparison.npy')
print(uq_results)

[[0.01474193 0.56980455 0.610897  ]
 [0.01225657 0.7444942  0.83405775]
 [0.01414623 0.7623601  0.84915298]
 [0.014622   0.75875348 0.84704018]]


## Dataset

In [4]:
import os
main = os.getcwd()

In [16]:
test = tf.data.Dataset.load(main+'/test_ds/')

In [17]:
test_len = len(test)
test_len

31

In [18]:
IMG_W = 256
IMG_H = 256
IMG_CH = 1
N_CLASSES = 5

def process(data):  
    img = data[0]
    msk = data[1]
    # img = tf.math.l2_normalize(img, axis=0)
    img = img/255

    msk = tf.squeeze(msk)
    msk = tf.cast(msk, tf.int32)
    msk = tf.one_hot(indices=msk, depth=N_CLASSES, axis=-1)

    img.set_shape([256,256,1])
    msk.set_shape([256,256,5])
    
    return img, msk

In [19]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
test = test.map(process, num_parallel_calls=AUTOTUNE)

In [20]:
BATCH_SIZE = 1

test = test.cache()
test = test.batch(BATCH_SIZE)
# test = test.prefetch(AUTOTUNE)

## Model

### Parameters

In [21]:
LR = 0.0001
EPOCHS = 100
optim = keras.optimizers.Adam(LR)
loss = keras.losses.categorical_crossentropy
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
focal_loss = sm.losses.BinaryFocalLoss() if N_CLASSES == 1 else sm.losses.CategoricalFocalLoss()

In [ ]:
model = models.att_unet_2d((256, 256, 1), filter_num=[64, 128, 256, 512, 1024], n_labels=5, stack_num_down=2, stack_num_up=2, activation='ReLU', 
                atten_activation='ReLU', attention='multiply', output_activation='Softmax', batch_norm=False, pool=True, unpool=True, 
                backbone=None, weights=None, freeze_backbone=False, freeze_batch_norm=True, name='attunet')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics =[tf.keras.metrics.MeanIoU(num_classes=5), sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)])

In [ ]:
# UNET Montecarlo Dropout

model = mc_unet_model(n_classes=5, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'mc_unet_model'
folder = 'MC_COMPARISON/UNET'
# model.summary()

In [ ]:
# UNET

model = unet_model(n_classes=5, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'unet_model'
folder = 'MC_COMPARISON/UNET'
# model.summary()

In [ ]:
# ATTN UNET PYRAMIDAL
model = att_unet(img_h=256, img_w=256, img_ch=1, n_label=5, data_format='channels_last')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'fpa_unet_model'
folder = 'FPA'
# model.summary()

In [ ]:
# ATTN UNET 
model = att_unet_org(img_h=256, img_w=256, img_ch=1, n_label=5, data_format='channels_last')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'attn_unet_model'
folder = 'MC_COMPARISON/ATTN'
# model.summary()

In [ ]:
# MC ATTN UNET 
model = mc_att_unet(img_h=256, img_w=256, img_ch=1, n_label=5, data_format='channels_last')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'mc_attn_unet_model'
folder = 'MC_COMPARISON/ATTN'
# model.summary()

In [ ]:
# Dense UNet 
model = dense_unet(input_shape=(256, 256, 1), num_classes=5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'dense_unet_model'
folder = 'MC_COMPARISON/DenseUNet'

In [ ]:
# MC Dense UNet 
model = mc_dense_unet(input_shape=(256, 256, 1), num_classes=5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'mc_dense_unet_model'
folder = 'MC_COMPARISON/DenseUNet'

In [ ]:
# Attn Dense UNet 
model = attn_dense_unet(input_shape=(256, 256, 1), num_classes=5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'attn_dense_unet_model'
folder = 'MC_COMPARISON/AttnDenseUNet'

In [16]:
# MC Attn Dense UNet 
model = mc_attn_dense_unet(input_shape=(256, 256, 1), num_classes=5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics = metrics)
name = 'mc_attn_dense_unet_model'
folder = 'MC_COMPARISON/AttnDenseUNet'

In [17]:
if not os.path.exists(folder):
    os.makedirs(folder)

# Callbacks
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, 
                                   patience=10, restore_best_weights=True)
log_dir ="logs/unet/fit/" + name+ '_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks= [es, tensorboard_callback, keras.callbacks.ModelCheckpoint(
                    filepath=(f'{folder}/{name}_best.hdf5'),
                    monitor='val_loss', save_best_only=True, mode='min')],

## Train

In [ ]:
#Training
history = model.fit(
    train, 
    epochs=EPOCHS, 
    callbacks=callbacks,
    validation_data=test)

# Generate generalization metrics
scores = model.evaluate(test, verbose=0)
print(f'Scores: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]};\
      {model.metrics_names[2]} of {scores[2]}')

### Send notification to iPhone ###
notif = requests.post(url=url_notif)

# serialize model to json
json_model = model.to_json()#save the model architecture to JSON file
with open(f'{folder}/{name}.json', 'w') as json_file:
    json_file.write(json_model)
    
model.save(f'{folder}/{name}.h5')
!sudo shutdown -h now


Epoch 1/100
159/262 [=================>............] - ETA: 16s - loss: 0.1937 - iou_score: 0.3033 - f1-score: 0.3082

## Load Model

In [22]:
import json
from tensorflow.keras.models import load_model, model_from_json
import segmentation_models as sm
# from mcdropout import MCDropout
from tensorflow.keras.layers import Dropout
name = 'unet_model'
folder = 'unet/'
with open(f'{folder}/{name}.json', 'r') as json_file:
    model = model_from_json(json_file.read(), custom_objects={"iou_score": sm.metrics.IOUScore(threshold=0.5),
                                                        "f1-score": sm.metrics.FScore(threshold=0.5), 
                                                        })
    


model.load_weights(f'{folder}/{name}.h5')
model.compile(optim, loss, metrics)

## Evaluation

In [23]:
result = model.evaluate(test)

31/31 [==============================] - 26s 821ms/step - loss: 0.1947 - iou_score: 0.4913 - f1-score: 0.5465
